In [2]:
import os.path as osp
import os
import glob
import torch
import awkward as ak
import time
import uproot
import uproot3
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.datasets import MNISTSuperpixels
from torch_geometric.data import DataListLoader, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import SplineConv, global_mean_pool, DataParallel, EdgeConv
from torch_geometric.data import Data
from torchsummary import summary
#from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.utils import to_categorical, plot_model
#from sklearn.neighbors import kneighbors_graph
import scipy.sparse as ss
from datetime import datetime, timedelta

In [4]:
def create_train_dataset_prmy(z, k, d, label):
    graphs = []
    for i in range(len(z)):
        vec = []
        vec.append(np.array([d[i], z[i], k[i]]).T)
        vec = np.array(vec)
        vec = np.squeeze(vec)
        ya = kneighbors_graph(vec, n_neighbors=int(np.floor(vec.shape[0]/2)))
        edges = np.array([ya.nonzero()[0], ya.nonzero()[1]])
        edge = torch.tensor(edges, dtype=torch.long)
        graphs.append(Data(x=torch.tensor(vec, dtype=torch.float), edge_index=edge, y=torch.tensor(label[i], dtype=torch.float)))
    return graphs

In [5]:
def create_train_dataset_fulld(z, k, d, p1, p2, label):
    graphs = []
    for i in range(len(z)):
        if i%1000 == 0: 
            print("Processing event {}/{}".format(i, len(z)), end="\r")
        vec = []
        vec.append(np.array([d[i], z[i], k[i]]).T)
        vec = np.array(vec)
        vec = np.squeeze(vec)

        v1 = [[ind, x] for ind, x in enumerate(p1[i]) if x > -1]
        v2 = [[ind, x] for ind, x in enumerate(p2[i]) if x > -1]

        a1 = np.reshape(v1,(len(v1),2)).T
        a2 = np.reshape(v2,(len(v2),2)).T
        edge1 = np.concatenate((a1[0], a2[0], a1[1], a2[1]),axis = 0)
        edge2 = np.concatenate((a1[1], a2[1], a1[0], a2[0]),axis = 0)
        edge = torch.tensor(np.array([edge1, edge2]), dtype=torch.long)
        graphs.append(Data(x=torch.tensor(vec, dtype=torch.float), edge_index=edge, y=torch.tensor(label[i], dtype=torch.float)))
    return graphs

In [6]:
def create_test_dataset_prmy(z, k, d):
    graphs = []
    for i in range(len(z)):
        vec = []
        vec.append(np.array([d[i], z[i], k[i]]).T)
        vec = np.array(vec)
        vec = np.squeeze(vec)
        ya = kneighbors_graph(vec, n_neighbors=int(np.floor(vec.shape[0]/2)))
        edges = np.array([ya.nonzero()[0], ya.nonzero()[1]])
        edge = torch.tensor(edges, dtype=torch.long)
        graphs.append(Data(x=torch.tensor(vec, dtype=torch.float), edge_index=edge))
    return graphs

In [7]:
def create_test_dataset_fulld(z, k, d, p1, p2):
    graphs = []
    for i in range(len(z)):
        vec = []
        vec.append(np.array([d[i], z[i], k[i]]).T)
        vec = np.array(vec)
        vec = np.squeeze(vec)
        v1 = [[ind, x] for ind, x in enumerate(p1[i]) if x > -1]
        v2 = [[ind, x] for ind, x in enumerate(p2[i]) if x > -1]

        a1 = np.reshape(v1,(len(v1),2)).T
        a2 = np.reshape(v2,(len(v2),2)).T
        edge1 = np.concatenate((a1[0], a2[0], a1[1], a2[1]),axis = 0)
        edge2 = np.concatenate((a1[1], a2[1], a1[0], a2[0]),axis = 0)
        edge = torch.tensor(np.array([edge1, edge2]), dtype=torch.long)
        graphs.append(Data(x=torch.tensor(vec, dtype=torch.float), edge_index=edge))
    return graphs

In [8]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = EdgeConv(nn.Sequential(nn.Linear(6, 64),
                                  nn.ReLU(), nn.Linear(64, 64), nn.ReLU(), nn.Linear(64, 64)),aggr='add')
        self.conv2 = EdgeConv(nn.Sequential(nn.Linear(128, 128),
                                  nn.ReLU(), nn.Linear(128, 128),nn.ReLU(), nn.Linear(128, 128)),aggr='add')
        self.conv3 = EdgeConv(nn.Sequential(nn.Linear(256,256,),
                                  nn.ReLU(), nn.Linear(256, 256),nn.ReLU(), nn.Linear(256, 256)),aggr='add')
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(256, 64)
        self.lin3 = torch.nn.Linear(64, 2)
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.1)
        x = self.lin2(x)
        x = F.dropout(x, p=0.1)
        x = self.lin3(x)
        #print(x.shape)
        return F.sigmoid(x)

In [13]:
class Adversary(nn.Module):
    def __init__(self):
        super(Adversary, self).__init__()
        self.lin1 = torch.nn.Linear(2, 32)
        self.lin2 = torch.nn.Linear(32, 64)
        self.lin3 = torch.nn.Linear(64, 128)
        self.lin4 = torch.nn.Linear(128, 256)
        
    def forward(self, data):
        x = data
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)
        x = self.lin4(x)
        return x

In [15]:
lr = 0.0002
beta_1 = 0.5
beta_2 = 0.999
device = torch.device('cuda:2')
gen = Net().to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr, betas=(beta_1, beta_2))
adv = Adversary().to(device)
adv_opt = torch.optim.Adam(adv.parameters(), lr=lr, betas=(beta_1, beta_2))

In [ ]:
n_epochs = 50
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
# I still have to work on this,
# it's just a GAN so not really a proper adversarial network
# I have to implement a Gradient Reversal Layer like in the adversarial package from
# the looks of it which will take a bit of time
for epoch in range(n_epochs):
    # Dataloader returns the batches
    for real in dataloader:
        cur_batch_size = len(real)
        real = real.to(device)

        ## Update discriminator ##
        disc_opt.zero_grad()
        fake_noise = get_noise(cur_batch_size, z_dim, device=device)
        fake = gen(fake_noise)
        disc_fake_pred = disc(fake.detach())
        disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))
        disc_real_pred = disc(real)
        disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred))
        disc_loss = (disc_fake_loss + disc_real_loss) / 2

        # Keep track of the average discriminator loss
        mean_discriminator_loss += disc_loss.item() / display_step
        # Update gradients
        disc_loss.backward(retain_graph=True)
        # Update optimizer
        disc_opt.step()

        ## Update generator ##
        gen_opt.zero_grad()
        fake_noise_2 = get_noise(cur_batch_size, z_dim, device=device)
        fake_2 = gen(fake_noise_2)
        disc_fake_pred = disc(fake_2)
        gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
        gen_loss.backward()
        gen_opt.step()

        # Keep track of the average generator loss
        mean_generator_loss += gen_loss.item() / display_step

        ## Visualization code ##
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1


In [9]:
#Configuration from bash script
if "INFILE" in os.environ:
    infile_path = os.environ["INFILE"]
    model_name = os.environ["MODEL"]
    epochs = int(os.environ["EPOCHS"])

#Configuration in notebook
else:    
#    files = glob.glob("/mnt/storage/asopio/train_test_split_20210305/training_set_jz2to9.root")
    files = glob.glob("/mnt/storage/raresiora/train_test_split/full_decluster/zprime/*_train.root") + glob.glob("/mnt/storage/raresiora/train_test_split/full_decluster/j3to9/*_train.root")
    model_name = "GNN_full"
#    model_name = "LSTM"
#    model_name = "1DCNN"
#    model_name = "2DCNN"
#    model_name = "ImgCNN"
#    model_name = "GNN"
#    model_name = "Transformer"
#    nb_epochs = 20

In [10]:
file1 = glob.glob("/mnt/storage/raresiora/train_test_split/primary_plane/wprime/*_train.root")
file2 = glob.glob("/mnt/storage/raresiora/train_test_split/primary_plane/zprime/*_train.root")
file3 = glob.glob("/mnt/storage/raresiora/train_test_split/primary_plane/j3to9/*_train.root")
nentries_total = 0
intreename = "lundjets_InDetTrackParticles"
print(files[0])
for infile_name in files: 
    nentries_total += uproot3.numentries(infile_name, intreename)

print("Evaluating on {} files with {} entries in total.".format(len(file3), nentries_total))

/mnt/storage/raresiora/train_test_split/full_decluster/zprime/user.asopio.24603668._000012.ANALYSIS.root_train.root
Evaluating on 16 files with 1066593 entries in total.


In [11]:
#Load tf keras model
# jet_type = "Akt10RecoChargedJet" #track jets
jet_type = "Akt10UFOJet" #UFO jets

save_trained_model = True
intreename = "lundjets_InDetTrackParticles"

model_filename = "save/models/"+model_name+".hdf5"

Full declustering data loading

In [12]:
def pad_ak(arr, l):
    arr = ak.pad_none(arr, l)
    arr = ak.fill_none(arr, 0)
    arr = ak.to_numpy(arr)
    return arr

def pad_ak3(arr, l):
    arr = ak.pad_none(arr, 1)
    arr = ak.fill_none(arr, [0])
    arr = arr[:,0]
    arr = ak.pad_none(arr, l)
    arr = ak.fill_none(arr, 0)
    #arr = ak.to_numpy(arr)
    return arr

print("Training tagger on files", len(files))
t_start = time.time()

dsids = np.array([])
NBHadrons = np.array([])
trial = np.ones((1,30))
all_lund_zs = ak.from_numpy(trial)
all_lund_kts = ak.from_numpy(trial)
all_lund_drs = ak.from_numpy(trial)
parent1 = ak.from_numpy(trial)
parent2 = ak.from_numpy(trial)
vector = []
for file in files:
    
    print("Loading file",file)
    
    infile = uproot.open(file)
    tree = infile[intreename]
    dsids = np.append( dsids, np.array(tree["DSID"].array()) )
    mcweights = tree["mcWeight"].array()
    NBHadrons = np.append( NBHadrons, pad_ak(tree["Akt10TruthJet_inputJetGABHadrons"].array(), 30)[:,0] )
    parent1 = ak.concatenate((parent1, pad_ak3(tree["{}_jetLundIDParent1".format(jet_type)].array(), 2)), axis = 0)
    parent2 = ak.concatenate((parent2, pad_ak3(tree["{}_jetLundIDParent2".format(jet_type)].array(), 2)), axis = 0)
    #Get jet kinematics
    
    all_lund_zs = ak.concatenate((all_lund_zs, pad_ak3(tree["{}_jetLundZ".format(jet_type)].array(), 2)), axis=0)
    all_lund_kts = ak.concatenate((all_lund_kts, pad_ak3(tree["{}_jetLundKt".format(jet_type)].array(), 2)), axis=0)
    all_lund_drs = ak.concatenate((all_lund_drs, pad_ak3(tree["{}_jetLundDeltaR".format(jet_type)].array(), 2)), axis=0)
    
all_lund_zs = all_lund_zs[1:]    
all_lund_kts = all_lund_kts[1:]
all_lund_drs = all_lund_drs[1:]
parent1 = parent1[1:]
parent2 = parent2[1:]

delta_t_fileax = time.time() - t_start
print("Opened data in {:.4f} seconds.".format(delta_t_fileax))

#Get labels
#labels = ( dsids > 360000 ) & ( dsids < 370000 )

labels = ( dsids > 370000 ) & ( NBHadrons > 0 ) # do NBHadrons == 0 for W bosons, NBHadrons > 0 for Tops


#print(labels)
labels = to_categorical(labels, 2)

Training tagger on files 20
Loading file /mnt/storage/raresiora/train_test_split/full_decluster/zprime/user.asopio.24603668._000012.ANALYSIS.root_train.root
Loading file /mnt/storage/raresiora/train_test_split/full_decluster/zprime/user.asopio.24603668._000005.ANALYSIS.root_train.root
Loading file /mnt/storage/raresiora/train_test_split/full_decluster/zprime/user.asopio.24603668._000019.ANALYSIS.root_train.root
Loading file /mnt/storage/raresiora/train_test_split/full_decluster/j3to9/user.asopio.24603633._000004.ANALYSIS.root_train.root
Loading file /mnt/storage/raresiora/train_test_split/full_decluster/j3to9/user.asopio.24603631._000012.ANALYSIS.root_train.root
Loading file /mnt/storage/raresiora/train_test_split/full_decluster/j3to9/user.asopio.24603627._000002.ANALYSIS.root_train.root
Loading file /mnt/storage/raresiora/train_test_split/full_decluster/j3to9/user.asopio.24603627._000008.ANALYSIS.root_train.root
Loading file /mnt/storage/raresiora/train_test_split/full_decluster/j3to9

Primary plane data loading

In [ ]:
# This part is self contained. I should rename the functions though.
# It works f
def pad_ak(arr, l):
    arr = ak.pad_none(arr, l)
    arr = ak.fill_none(arr, 0)
    arr = ak.to_numpy(arr)
    return arr

def pad_ak3(arr, l):
    arr = ak.pad_none(arr, 1)
    arr = ak.fill_none(arr, [0])
    arr = arr[:,0]
    arr = ak.pad_none(arr, l)
    arr = ak.fill_none(arr, 0)
    #arr = ak.to_numpy(arr)
    return arr

print("Training tagger on files", len(files))
t_start = time.time()
dsids = np.array([])
NBHadrons = np.array([])
trial = np.ones((1,30))
all_lund_zs = ak.from_numpy(trial)
all_lund_kts = ak.from_numpy(trial)
all_lund_drs = ak.from_numpy(trial)


for file in files:
    
    print("Loading file",file)
    infile = uproot.open(file)
    tree = infile[intreename]
    dsids = np.append( dsids, np.array(tree["DSID"].array()) )
    mcweights = tree["mcWeight"].array()
    NBHadrons = np.append( NBHadrons, pad_ak(tree["Akt10TruthJet_inputJetGABHadrons"].array(), 30)[:,0] )
    #Get jet kinematics
    
    #The following 4 variables are irrelevant for training, might as well not load them here if you don't want
    jet_pts = pad_ak(tree["AntiKt10UFOCSSKSoftDropBeta100Zcut10JetsCalib_jetPt"].array(), 30)[:,0]
    jet_etas = pad_ak(tree["AntiKt10UFOCSSKSoftDropBeta100Zcut10JetsCalib_jetEta"].array(), 30)[:,0]
    jet_phis = pad_ak(tree["AntiKt10UFOCSSKSoftDropBeta100Zcut10JetsCalib_jetPhi"].array(), 30)[:,0]              
    jet_ms = pad_ak(tree["AntiKt10UFOCSSKSoftDropBeta100Zcut10JetsCalib_jetM"].array(), 30)[:,0]

    all_lund_zs = ak.concatenate((all_lund_zs, pad_ak3(tree["{}_jetLundZ".format(jet_type)].array(), 2)), axis=0)
    all_lund_kts = ak.concatenate((all_lund_kts, pad_ak3(tree["{}_jetLundKt".format(jet_type)].array(), 2)), axis=0)
    all_lund_drs = ak.concatenate((all_lund_drs, pad_ak3(tree["{}_jetLundDeltaR".format(jet_type)].array(), 2)), axis=0)
    
all_lund_zs = all_lund_zs[1:]    
all_lund_kts = all_lund_kts[1:]
all_lund_drs = all_lund_drs[1:]
    
delta_t_fileax = time.time() - t_start
print("Opened data in {:.4f} seconds.".format(delta_t_fileax))

#Get labels
#labels = ( dsids > 360000 ) & ( dsids < 370000 )

labels = ( dsids > 370000 ) # depends on your signal and background definition


# print(labels)
labels = to_categorical(labels, 2)

In [12]:
# This finds the number of trainable parameters for your model
model = Net()
total = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(total)

305154


For Primary plane data

In [ ]:
#W bosons
dataset = create_train_dataset_prmy(all_lund_zs, all_lund_kts, all_lund_drs, labels) 
train_loader = DataLoader(dataset, batch_size=1024, shuffle=True)

In [ ]:
#Top Quark
dataset2 = create_train_dataset_prmy(all_lund_zs, all_lund_kts, all_lund_drs, labels)
train_loader = DataLoader(dataset2, batch_size=1024, shuffle=True)

For Full declustering data

In [11]:
#W bosons
# It will take about 30 minutes to finish
dataset = create_train_dataset_fulld(all_lund_zs, all_lund_kts, all_lund_drs, parent1, parent2, labels) 
train_loader = DataLoader(dataset, batch_size=1024, shuffle=True)

In [17]:
#Top Quark
# It will take about 25 minutes to finish
dataset = create_train_dataset_fulld(all_lund_zs, all_lund_kts, all_lund_drs, parent1, parent2, labels)
train_loader = DataLoader(dataset, batch_size=1024, shuffle=True)

In [18]:
print("Boom")

Boom


Model Training

In [19]:
model = Net()
device = torch.device('cuda:2') # Usually gpu 4 worked best, it had the most memory available
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

def train(epoch):
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        new_y = torch.reshape(data.y, (int(list(data.y.shape)[0]/2),2))
        loss = F.binary_cross_entropy(output, new_y)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(dataset)

@torch.no_grad()
def get_accuracy(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        new_y = torch.reshape(data.y, (int(list(data.y.shape)[0]/2),2))
        pred = model(data).max(dim=1)[1]
        correct += pred.eq(new_y[:,1]).sum().item()
    return correct / len(loader.dataset)
    
@torch.no_grad()
def get_scores(loader):
    model.eval()
    total_output = np.array([[1,1]])
    for data in loader:
        data = data.to(device)
        pred = model(data)
        total_output = np.append(total_output, pred.cpu().detach().numpy(), axis=0)
    return total_output[1:]
    
for epoch in range(1, 21):
    print("Epoch:{}".format(epoch))
    loss = train(epoch)
    train_acc = get_accuracy(train_loader)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Acc: {:.5f}'.format(epoch, loss, train_acc))

Epoch:1
Epoch: 001, Loss: 0.23922, Train Acc: 0.90895
Epoch:2
Epoch: 002, Loss: 0.21002, Train Acc: 0.90863
Epoch:3
Epoch: 003, Loss: 0.19958, Train Acc: 0.91428
Epoch:4
Epoch: 004, Loss: 0.19089, Train Acc: 0.92489
Epoch:5
Epoch: 005, Loss: 0.18531, Train Acc: 0.92552
Epoch:6
Epoch: 006, Loss: 0.18220, Train Acc: 0.91875
Epoch:7
Epoch: 007, Loss: 0.17972, Train Acc: 0.92899
Epoch:8
Epoch: 008, Loss: 0.17710, Train Acc: 0.92964
Epoch:9
Epoch: 009, Loss: 0.17598, Train Acc: 0.92867
Epoch:10
Epoch: 010, Loss: 0.17500, Train Acc: 0.92726
Epoch:11
Epoch: 011, Loss: 0.17329, Train Acc: 0.93061
Epoch:12
Epoch: 012, Loss: 0.17289, Train Acc: 0.92954
Epoch:13
Epoch: 013, Loss: 0.17114, Train Acc: 0.92458
Epoch:14
Epoch: 014, Loss: 0.17042, Train Acc: 0.93188
Epoch:15
Epoch: 015, Loss: 0.16937, Train Acc: 0.93072
Epoch:16
Epoch: 016, Loss: 0.16916, Train Acc: 0.93206
Epoch:17
Epoch: 017, Loss: 0.16708, Train Acc: 0.93219
Epoch:18
Epoch: 018, Loss: 0.16643, Train Acc: 0.93343
Epoch:19
Epoch: 019

In [20]:
path = "/your/path/to/models/zprime.pt"

In [21]:
torch.save(model.state_dict(), path)

# Evaluation part

In [22]:
#output ntuple directory
# outdir = "/mnt/storage/asopio"
outdir = "your/path/to/scores"
model_name = "GNN_fulld_20210523_t"
#input TTree name
intreename = "lundjets_InDetTrackParticles"

#infiles_list = glob.glob(infiles_bkg) + glob.glob(infiles_sig)
#infiles_list = glob.glob("/mnt/storage/asopio/train_test_split_20210305/*ANALYSIS.root_test.root")
#file1 = glob.glob("/mnt/storage/asopio/train_test_split_20210305/wprime/*ANALYSIS.root_test.root")
#file2 = glob.glob("/mnt/storage/asopio/train_test_split_20210305/jz2/*ANALYSIS.root_test.root")
#file3 = glob.glob("/mnt/storage/asopio/train_test_split_20210305/jz3/*ANALYSIS.root_test.root")
#file4 = glob.glob("/mnt/storage/asopio/train_test_split_20210305/jz4/*ANALYSIS.root_test.root")
#file5 = glob.glob("/mnt/storage/asopio/train_test_split_20210305/jz5/*ANALYSIS.root_test.root")
#file6 = glob.glob("/mnt/storage/asopio/train_test_split_20210305/jz6/*ANALYSIS.root_test.root")
#file7 = glob.glob("/mnt/storage/asopio/train_test_split_20210305/jz7/*ANALYSIS.root_test.root")
#file8 = glob.glob("/mnt/storage/asopio/train_test_split_20210305/jz8/*ANALYSIS.root_test.root")
#file9 = glob.glob("/mnt/storage/asopio/train_test_split_20210305/jz9/*ANALYSIS.root_test.root")
#infiles_list = file1 + file3 + file4 + file5 + file6 + file7 + file8 + file9
infiles_list = glob.glob("/mnt/storage/raresiora/train_test_split/full_decluster/zprime/*_test.root") + glob.glob("/mnt/storage/raresiora/train_test_split/full_decluster/j3to9/*_test.root")
#files = glob.glob("/mnt/storage/raresiora/train_test_split/full_decluster/wprime/*_train.root") + glob.glob("/mnt/storage/raresiora/train_test_split/full_decluster/j3to9/*_train.root")
nentries_total = 0
for infile_name in infiles_list: 
    nentries_total += uproot3.numentries(infile_name, intreename)

print("Evaluating on {} files with {} entries in total.".format(len(infiles_list), nentries_total))

Evaluating on 20 files with 9599237 entries in total.


In [23]:
nentries = 0
print(model_name)
t_start = time.time()
for i_file, infile_name in enumerate(infiles_list):
    
    print("Loading file {}/{}".format(i_file+1, len(infiles_list)))
    t_filestart = time.time()
    
    #Open file 
    file = uproot.open(infile_name)
    tree = file[intreename]    
    
    #Get weights info
    dsids = tree["DSID"].array()
    mcweights = tree["mcWeight"].array()
    NBHadrons = pad_ak(tree["Akt10TruthJet_inputJetGABHadrons"].array(), 30)[:,0]
    #### 
    #For Full decluster
    
    parent1 = pad_ak3(tree["Akt10UFOJet_jetLundIDParent1"].array(), 2)
    parent2 = pad_ak3(tree["Akt10UFOJet_jetLundIDParent2"].array(), 2)
    print(len(parent1))
    ####
    #Get jet kinematics
    jet_pts = pad_ak(tree["AntiKt10UFOCSSKSoftDropBeta100Zcut10JetsCalib_jetPt"].array(), 30)[:,0]
    jet_etas = pad_ak(tree["AntiKt10UFOCSSKSoftDropBeta100Zcut10JetsCalib_jetEta"].array(), 30)[:,0]
    jet_phis = pad_ak(tree["AntiKt10UFOCSSKSoftDropBeta100Zcut10JetsCalib_jetPhi"].array(), 30)[:,0]                      
    jet_ms = pad_ak(tree["AntiKt10UFOCSSKSoftDropBeta100Zcut10JetsCalib_jetM"].array(), 30)[:,0]
    
    #Get lund values
    all_lund_zs = pad_ak3(tree["{}_jetLundZ".format(jet_type)].array(), 2)
    all_lund_kts = pad_ak3(tree["{}_jetLundKt".format(jet_type)].array(), 2)
    all_lund_drs = pad_ak3(tree["{}_jetLundDeltaR".format(jet_type)].array(), 2)
    print(len(all_lund_zs))
    delta_t_fileax = time.time() - t_filestart
    print("Opened data in {:.4f} seconds.".format(delta_t_fileax))
     
    #### 
    #For Full decluster
    
    test_data = create_test_dataset_fulld(all_lund_zs, all_lund_kts, all_lund_drs, parent1, parent2)
    
    ####
    
    #test_data = create_test_dataset_prmy(all_lund_zs, all_lund_kts, all_lund_drs)
    test_loader = DataLoader(test_data, batch_size=1024, shuffle=False)
    
    #Predict scores
    y_pred = get_scores(test_loader)
    print(y_pred)
    tagger_scores = y_pred[:,1]
    inv_tagger_scores = y_pred[:,0]
    delta_t_pred = time.time() - t_filestart - delta_t_fileax
    print("Calculated predicitions in {:.4f} seconds,".format(delta_t_pred))

    #Save root files containing model scores
    filename = infile_name.split("/")[-1]
    outfile_path = os.path.join(outdir, filename) 
    
    
    with uproot3.recreate("{}_score_{}.root".format(outfile_path, model_name)) as f:

        treename = "FlatSubstructureJetTree"

        #Declare branch data types
        f[treename] = uproot3.newtree({"EventInfo_mcChannelNumber": "int32",
                                      "EventInfo_mcEventWeight": "float32",
                                      "EventInfo_NBHadrons": "int32",   # I doubt saving the parents is necessary here
                                      "fjet_nnscore": "float32",        # which is why I didn't include them
                                      "fjet_invnnscore": "float32", 
                                      "fjet_pt": "float32",
                                      "fjet_eta": "float32",
                                      "fjet_phi": "float32",
                                      "fjet_m": "float32",
                                      })

        #Save branches
        f[treename].extend({"EventInfo_mcChannelNumber": dsids,
                            "EventInfo_mcEventWeight": mcweights,
                            "EventInfo_NBHadrons": NBHadrons,
                            "fjet_nnscore": tagger_scores, 
                            "fjet_invnnscore": inv_tagger_scores, 
                            "fjet_pt": jet_pts,
                            "fjet_eta": jet_etas,
                            "fjet_phi": jet_phis,
                            "fjet_m": jet_ms,
                            })

    delta_t_save = time.time() - t_start - delta_t_fileax - delta_t_pred
    print("Saved data in {:.4f} seconds.".format(delta_t_save))
    
    
    #Time statistics
    nentries += uproot3.numentries(infile_name, intreename)
    time_per_entry = (time.time() - t_start)/nentries
    eta = time_per_entry * (nentries_total - nentries)
    
    print("Evaluated on {} out of {} events".format(nentries, nentries_total))    
    print("Estimated time until completion: {}".format(str(timedelta(seconds=eta))))
    
    
print("Total evaluation time: {:.4f} seconds.".format(time.time()-t_start))

GNN_fulld_20210523_t
Loading file 1/20
609645
609645
Opened data in 2621.6617 seconds.
[[0.38414103 0.61684531]
 [0.99529809 0.00476372]
 [0.94582957 0.05464448]
 ...
 [0.38517058 0.61635953]
 [0.83747154 0.16330679]
 [0.97318876 0.02709857]]
Calculated predicitions in 1214.9363 seconds,
Saved data in 2.4441 seconds.
Evaluated on 609645 out of 9599237 events
Estimated time until completion: 15:43:29.218569
Loading file 2/20
631579
631579
Opened data in 2510.6316 seconds.
[[0.335794   0.66515303]
 [0.90862072 0.09212638]
 [0.43054235 0.56968766]
 ...
 [0.00643208 0.99310815]
 [0.36816323 0.63195419]
 [0.02088805 0.97785711]]
Calculated predicitions in 1257.9484 seconds,
Saved data in 3840.0408 seconds.
Evaluated on 1241224 out of 9599237 events
Estimated time until completion: 14:13:54.143969
Loading file 3/20
186480
186480
Opened data in 745.8780 seconds.
[[0.99894911 0.00103862]
 [0.49387866 0.50608742]
 [0.86327875 0.13911542]
 ...
 [0.1750643  0.82536817]
 [0.65403146 0.3457523 ]
 [

Saved data in 49610.9466 seconds.
Evaluated on 9599237 out of 9599237 events
Estimated time until completion: 0:00:00
Total evaluation time: 50778.9891 seconds.
